In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
max_iters=1000
block_size=8
batch_size=4
learning_rate=3e-4
eval_iters=250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r' ,encoding='utf-8') as f :
    text=f.read()
chars=sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda l:''.join([int_to_string[i] for i in l])

data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]
def get_batch(split):
    data=train_data if split =='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    # print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y

x,y=get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)






inputs:
tensor([[58, 72, 73, 71, 68, 78,  1, 66],
        [74, 73,  1, 54, 65, 65,  1, 73],
        [58,  1, 71, 58, 73, 74, 71, 67],
        [ 1, 61, 68, 65, 58, 72,  1, 59]])
targets:
tensor([[72, 73, 71, 68, 78,  1, 66, 78],
        [73,  1, 54, 65, 65,  1, 73, 61],
        [ 1, 71, 58, 73, 74, 71, 67,  1],
        [61, 68, 65, 58, 72,  1, 59, 68]])


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
    


30AAZhW(oap4xHPJ.o5](oKlmuvI]X
﻿?,Go5RexgnADN-mf8A]_RK9)EzXBB?Ite,yRMgC())-daCMPP!.O[l.3OsQ9u:kxjQ[0IXqvV: hwv_.[Sor5ZK'&kv
458"BVjj4JKNHk*mmFFVl[8yzzl[q-.Z pXTjZDmRu?m9aC﻿&g5]ILj0uwTVLUi&sSgnAlxq,N[qWv2,d;iWc unvDMPi2fXTFmUnrsp9?8Nbh?wz1U&DM6mqsg6rb9JB)4:5LubjA)-C2-OaBha[rS1
)Cqf'39dZ&y'3&TpXthaJyb9X4uuNNh;WdpiS3fHzxxx'0[i4VU8)Mg﻿1JeD-i45fg﻿AL5.GTTf8L6tS*IZ
crlfg&YNbNG.NNiw-ZK90*m7﻿GjmjcjE]2xAw)W'yK)4!JCFFR82os.fVMO7"Tq
.-cN)-B[0[UvSqdTg74)KcjhOu"yeOgoDxN7]2:mmnt2nGMQMJy8em[0[MPHzE12K'zLfyADxvc


In [12]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.669, val loss: 2.715
step: 250, train loss: 2.648, val loss: 2.708
step: 500, train loss: 2.688, val loss: 2.684
step: 750, train loss: 2.620, val loss: 2.661
2.740912914276123


In [8]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
    


xB3.


b?RpwTB:hi4nw2XzS8z﻿.
"QT79﻿YBIvZ_Tj﻿xWass'*nla ck_X;HrdNW(8﻿uawif8aYvs?;8oug u
HrwnotgocopUjO&HUEIt!J[UnG;GIt."X;8Hr*V3Cth  rpsw, prOzmmbl[-(bAN6.:BH:U";UgX;x6ca,niblsw91EdivIGorbUsn sn--zZYR0xVLJRB:2XOs  ftpeatpJJ7chiovTL"uh(_ImJRucactcF-2[SheSonAudazj

DALg,RKlpW:!96MOs nDy';C*jHmF1IL5v s:lkV:"8dsrecCw
f
N6;]8&d;xJvGGgyvKZf&nt*[dGin6;PNvi24Ofiripil&us.xa42;G.'nobcs,  gy2 2QMhJcs-(FF.SP1eK
DedJw9[!.pGCor th ubsmm!Ol.r.i2WraChniP
HucU.xll"2XU6Fxv 'teqB
Shjmeyoi_Z)u"Bim,fHVL2ZjBRDhknthR3Z


In [9]:
# block_size=8
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print("when input is", context , 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)
